In [2]:
import random
import json
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [3]:
lemmatizer = WordNetLemmatizer()
json_file_path= '***********.json file path***********'
intents = json.loads(open(json_file_path).read())

In [4]:
words_pkl_path="words.pkl"
classes_pkl_path='classes.pkl'
model_path='chatbot_model.h5'

words=pickle.load(open(words_pkl_path, "rb"))
classes= pickle.load(open(classes_pkl_path, 'rb'))
model= load_model(model_path)

In [5]:
def clean_up_sentence(sentence):
    sentence_words= nltk.word_tokenize(sentence)
    sentence_words= [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_words(sentence)
    res = model.predict(np.array([bow]))[0]
    error_threshold= 0.25
    result= [[i,r] for i,r in enumerate(res) if r>error_threshold]
    
    result.sort(key=lambda x:x[1], reverse=True)
    return_list=[]
    for r in result:
        return_list.append({'intent': classes[r[0]], 'probability':str(r[1])})
    return return_list
def get_response(intents_list, intents_json):
    tag= intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result= random.choice(i['responses'])
            break
    return result

# Gui

In [8]:
import tkinter as tk

In [9]:
def send_message():
    message = user_input.get()
    chat_box.config(state=tk.NORMAL)
    chat_box.insert(tk.END, "You: " + message + "\n")
    chat_box.config(state=tk.DISABLED)
    user_input.delete(0, tk.END)  # Clear the input field
        # Generate and display bot's response
    bot_response = generate_bot_response(message)
    chat_box.config(state=tk.NORMAL)
    chat_box.insert(tk.END, "Bot: " + bot_response + "\n")
    chat_box.config(state=tk.DISABLED)

    
def generate_bot_response(user_message):
    ints=predict_class(user_message)
    res=get_response(ints,intents)
    return res

In [10]:
# Create the main window
root = tk.Tk()
root.title("Chat Bot")

# Create a frame for the chat history
chat_frame = tk.Frame(root)
chat_frame.pack(pady=10)

# Create a scrollable text widget to display the chat history
chat_box = tk.Text(chat_frame, height=20, width=50)
chat_box.pack(side=tk.LEFT, fill=tk.BOTH, padx=10, pady=10)
chat_box.config(state=tk.DISABLED)  # Disable editing

# Create a scrollbar for the chat history
scrollbar = tk.Scrollbar(chat_frame, command=chat_box.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
chat_box.config(yscrollcommand=scrollbar.set)

# Create a frame for the input field and send button
input_frame = tk.Frame(root)
input_frame.pack(pady=10)

# Create an entry widget for user input
user_input = tk.Entry(input_frame, width=40)
user_input.pack(side=tk.LEFT, padx=10)

# Create a button to send messages
send_button = tk.Button(input_frame, text="Send", width=10, command=send_message)
send_button.pack(side=tk.RIGHT)

# Start the main event loop
root.mainloop()


1/1 [==============================] - 0s 30ms/step
